In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 100
num_epochs = 2

In [20]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn_layer = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.nn_layer1 = nn.Linear(hidden_size*sequence_length, num_classes)
    
    def forward(self, x):
        # Declaring a hidden state initially
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # Ignoring the next hidden state received
        x, _ = self.rnn_layer(x, h0)
        x = x.reshape(x.shape[0], -1)
        x = self.nn_layer1(x)
        return x

In [21]:
train_dataset = datasets.MNIST(root='Data/MNIST-Dataset', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='Data/MNIST-Dataset', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [22]:
model = RNN(input_size, hidden_size, num_layers, num_classes)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
model.train()

for epoch in range(num_epochs):
    num_samples = 0
    num_correct = 0
    for batch_idx, (data, targets) in enumerate(train_loader):

        # The Dataset gives Nx1x28x28, (1 for the color channel, N for the batch_size)
        # But the RNN expects Nx28x28. Therefore, removing the 1 (column of the color channel)
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)
        
        predicted_output = model(data)
        
        _, prediction = predicted_output.max(1)
        num_samples += predicted_output.size(0)
        num_correct += (prediction==targets).sum()
        
        loss = loss_function(predicted_output, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if ((batch_idx+1) % 200 == 0):
            print(f'Batch {batch_idx+1}: Accuracy: {num_correct / num_samples * 100.00}')
    print(f'Epoch {epoch+1} completed with Accuracy {num_correct / num_samples * 100.00}')
    print()
print()
print("Training Completed!")
   

Batch 200: Accuracy: 95.54000091552734
Batch 400: Accuracy: 95.77749633789062
Batch 600: Accuracy: 95.99000549316406
Epoch 1 completed with Accuracy 95.99000549316406

Batch 200: Accuracy: 96.28499603271484
Batch 400: Accuracy: 96.59249877929688
Batch 600: Accuracy: 96.74333190917969
Epoch 2 completed with Accuracy 96.74333190917969


Training Completed!


In [28]:
# Model Testing
def check_accuracy(loader, model):
    model.eval() # Set model into evaluation mode!
    num_samples = 0
    num_correct = 0
    for data, targets in loader:
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)
        
        predicted_output = model(data)
        
        _, prediction = predicted_output.max(1)
        num_samples += predicted_output.size(0)
        num_correct += (prediction==targets).sum()

    print(f'Correctly identified samples: {num_correct}')
    print(f'Total samples: {num_samples}')
    print(f'The Validation Accuracy is {num_correct / num_samples * 100.00:.2f}')
    model.train() # Setting model back to training data

In [29]:
print("Checking Accuracy on Training Data...")
check_accuracy(train_loader, model)
print()
print("Checking Accuracy on Testing Data...")
check_accuracy(test_loader, model)

Checking Accuracy on Training Data...
Correctly identified samples: 58169
Total samples: 60000
The Validation Accuracy is 96.95

Checking Accuracy on Testing Data...
Correctly identified samples: 9679
Total samples: 10000
The Validation Accuracy is 96.79


In [31]:
# GRU: Gated Neural Network

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru_layer = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.nn_layer1 = nn.Linear(hidden_size*sequence_length, num_classes)
    
    def forward(self, x):
        # Declaring a hidden state initially
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # Ignoring the next hidden state received
        x, _ = self.gru_layer(x, h0)
        x = x.reshape(x.shape[0], -1)
        x = self.nn_layer1(x)
        return x
    
model = GRU(input_size, hidden_size, num_layers, num_classes)

model.train()

for epoch in range(num_epochs):
    num_samples = 0
    num_correct = 0
    for batch_idx, (data, targets) in enumerate(train_loader):

        # The Dataset gives Nx1x28x28, (1 for the color channel, N for the batch_size)
        # But the RNN expects Nx28x28. Therefore, removing the 1 (column of the color channel)
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)
        
        predicted_output = model(data)
        
        _, prediction = predicted_output.max(1)
        num_samples += predicted_output.size(0)
        num_correct += (prediction==targets).sum()
        
        loss = loss_function(predicted_output, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if ((batch_idx+1) % 200 == 0):
            print(f'Batch {batch_idx+1}: Accuracy: {num_correct / num_samples * 100.00}')
    print(f'Epoch {epoch+1} completed with Accuracy {num_correct / num_samples * 100.00}')
    print()
print()
print("Training Completed!")
print()
print("Checking Accuracy on Training Data...")
check_accuracy(train_loader, model)
print()
print("Checking Accuracy on Testing Data...")
check_accuracy(test_loader, model)

Batch 200: Accuracy: 10.84999942779541
Batch 400: Accuracy: 10.774999618530273
Batch 600: Accuracy: 10.773333549499512
Epoch 1 completed with Accuracy 10.773333549499512

Batch 200: Accuracy: 10.829999923706055
Batch 400: Accuracy: 10.84999942779541
Batch 600: Accuracy: 10.773333549499512
Epoch 2 completed with Accuracy 10.773333549499512


Training Completed!

Checking Accuracy on Training Data...
Correctly identified samples: 6464
Total samples: 60000
The Validation Accuracy is 10.77

Checking Accuracy on Testing Data...
Correctly identified samples: 1118
Total samples: 10000
The Validation Accuracy is 11.18


In [33]:
# LSTM: Long Short Term Memory

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm_layer = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.nn_layer1 = nn.Linear(hidden_size*sequence_length, num_classes)
    
    def forward(self, x):
        # Declaring a hidden state initially
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # Ignoring the next hidden state received
        x, _ = self.lstm_layer(x, (h0, c0))
        x = x.reshape(x.shape[0], -1)
        x = self.nn_layer1(x)
        return x
    
model = LSTM(input_size, hidden_size, num_layers, num_classes)

model.train()

for epoch in range(num_epochs):
    num_samples = 0
    num_correct = 0
    for batch_idx, (data, targets) in enumerate(train_loader):

        # The Dataset gives Nx1x28x28, (1 for the color channel, N for the batch_size)
        # But the RNN expects Nx28x28. Therefore, removing the 1 (column of the color channel)
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)
        
        predicted_output = model(data)
        
        _, prediction = predicted_output.max(1)
        num_samples += predicted_output.size(0)
        num_correct += (prediction==targets).sum()
        
        loss = loss_function(predicted_output, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if ((batch_idx+1) % 200 == 0):
            print(f'Batch {batch_idx+1}: Accuracy: {num_correct / num_samples * 100.00}')
    print(f'Epoch {epoch+1} completed with Accuracy {num_correct / num_samples * 100.00}')
    print()
print()
print("Training Completed!")
print()
print("Checking Accuracy on Training Data...")
check_accuracy(train_loader, model)
print()
print("Checking Accuracy on Testing Data...")
check_accuracy(test_loader, model)

Batch 200: Accuracy: 8.59000015258789
Batch 400: Accuracy: 8.6850004196167
Batch 600: Accuracy: 8.806666374206543
Epoch 1 completed with Accuracy 8.806666374206543

Batch 200: Accuracy: 8.675000190734863
Batch 400: Accuracy: 8.710000038146973
Batch 600: Accuracy: 8.806666374206543
Epoch 2 completed with Accuracy 8.806666374206543


Training Completed!

Checking Accuracy on Training Data...
Correctly identified samples: 5284
Total samples: 60000
The Validation Accuracy is 8.81

Checking Accuracy on Testing Data...
Correctly identified samples: 917
Total samples: 10000
The Validation Accuracy is 9.17
